In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd

In [2]:
def load_netcdf_files(directory):
    files = [f for f in os.listdir(directory) if f.endswith('.nc')]
    datasets = [xr.open_dataset(os.path.join(directory, f)) for f in files]
    return datasets


In [ ]:
def merge_per_cycle(directory):
    datasets = load_netcdf_files(directory)
    
    merged = None

    for ds in datasets:
        if merged is None:
            merged = ds
        elif len(ds.iOBS) > 0:
            merged = xr.concat([merged, ds], dim='iOBS')

    return merged

In [43]:
directory9 = '/nobackup/sreich/swot/swot_obsfit_L3/cycle_009'
directory10 = '/nobackup/sreich/swot/swot_obsfit_L3/cycle_010'
#datasets = load_netcdf_files(directory)
#datasets

In [46]:
merged9 = merge_per_cycle(directory9)


In [ ]:
merged10 = merge_per_cycle(directory10)

In [49]:
ds_all = xr.concat([merged9, merged10], dim='iOBS')
ds_all

<xarray.Dataset>
Dimensions:               (iOBS: 41441414)
Coordinates:
    longitude             (iOBS) float64 -75.12 -75.08 -75.05 ... 78.2 78.23
    latitude              (iOBS) float64 -25.47 -25.47 -25.4 ... 60.12 60.03
Dimensions without coordinates: iOBS
Data variables: (12/15)
    obs_date              (iOBS) float64 7.393e+05 7.393e+05 ... 7.393e+05
    obs_YYYYMMDD          (iOBS) int64 20240106 20240106 ... 20240213 20240213
    obs_HHMMSS            (iOBS) int64 15510 15511 15511 ... 233825 233825
    sample_x              (iOBS) float64 -75.12 -75.08 -75.05 ... 78.2 78.23
    sample_y              (iOBS) float64 -25.47 -25.47 -25.4 ... 60.12 60.03
    sample_z              (iOBS) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...                    ...
    sample_interp_XC11    (iOBS) float64 -38.56 -38.56 -38.56 ... 52.0 52.0 52.0
    sample_interp_YC11    (iOBS) float64 -76.22 -76.22 -76.22 ... 74.06 74.06
    sample_interp_XCNINJ  (iOBS) float64 -23.04 -23.04 -23.04 ... 52.0 52.0 52.0
    sample_interp_YCNINJ  (iOBS) float64 -70.02 -70.02 -70.02 ... 82.09 82.09
    sample_interp_i       (iOBS) float64 151.0 151.0 152.0 ... 132.0 133.0 133.0
    sample_interp_j       (iOBS) float64 37.0 38.0 39.0 46.0 ... 61.0 60.0 61.0

In [53]:
obs = xr.Dataset(
    data_vars=dict(
        obs_date           =(["iOBS"], ds_all.obs_date.values),
        obs_YYYYMMDD       =(["iOBS"], ds_all.obs_YYYYMMDD.values),
        obs_HHMMSS         =(["iOBS"], ds_all.obs_HHMMSS.values), 
        sample_x           =(["iSAMPLE"], ds_all.sample_x.values),
        sample_y           =(["iSAMPLE"], ds_all.sample_y.values),
        sample_z           =(["iSAMPLE"], ds_all.sample_z.values),
        sample_type        =(["iSAMPLE"], ds_all.sample_type.values),
        obs_val            =(["iOBS"], ds_all.obs_val.values),
        obs_uncert         =(["iOBS"], np.ones(len(ds_all.sample_interp_i))*0.02),
        sample_interp_XC11 =(["iOBS"], ds_all.sample_interp_XC11.values ),
        sample_interp_YC11 =(["iOBS"], ds_all.sample_interp_YC11.values ),
        sample_interp_XCNINJ =(["iOBS"], ds_all.sample_interp_XCNINJ.values ),
        sample_interp_YCNINJ =(["iOBS"], ds_all.sample_interp_YCNINJ.values ),
        sample_interp_i =(["iOBS"], ds_all.sample_interp_i.values ),
        sample_interp_j =(["iOBS"], ds_all.sample_interp_j.values ),
        sample_interp_w =(["iOBS", "iINTERP"], np.ones((len(ds_all.sample_interp_i),8))/8  )
    ),
)

In [54]:
obs

<xarray.Dataset>
Dimensions:               (iOBS: 41441414, iSAMPLE: 41441414, iINTERP: 8)
Dimensions without coordinates: iOBS, iSAMPLE, iINTERP
Data variables: (12/16)
    obs_date              (iOBS) float64 7.393e+05 7.393e+05 ... 7.393e+05
    obs_YYYYMMDD          (iOBS) int64 20240106 20240106 ... 20240213 20240213
    obs_HHMMSS            (iOBS) int64 15510 15511 15511 ... 233825 233825
    sample_x              (iSAMPLE) float64 -75.12 -75.08 -75.05 ... 78.2 78.23
    sample_y              (iSAMPLE) float64 -25.47 -25.47 -25.4 ... 60.12 60.03
    sample_z              (iSAMPLE) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0
    ...                    ...
    sample_interp_YC11    (iOBS) float64 -76.22 -76.22 -76.22 ... 74.06 74.06
    sample_interp_XCNINJ  (iOBS) float64 -23.04 -23.04 -23.04 ... 52.0 52.0 52.0
    sample_interp_YCNINJ  (iOBS) float64 -70.02 -70.02 -70.02 ... 82.09 82.09
    sample_interp_i       (iOBS) float64 151.0 151.0 152.0 ... 132.0 133.0 133.0
    sample_interp_j       (iOBS) float64 37.0 38.0 39.0 46.0 ... 61.0 60.0 61.0
    sample_interp_w       (iOBS, iINTERP) float64 0.125 0.125 ... 0.125 0.125

In [55]:
data_dir = '/nobackup/sreich/swot/L3_aviso/'
fname =  'swot_cycles_009_010_obsfit.nc'
obs.to_netcdf(data_dir + fname)